In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
#import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import ensemble
import time
import pickle
import random
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import OneHotEncoder
import re
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
from keras.layers import RepeatVector
from keras.layers import Permute
from keras.layers import merge
from keras.layers import Lambda
from keras.layers import multiply
from keras.layers import concatenate
from keras.layers import Convolution1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import GlobalMaxPooling1D
from sklearn import preprocessing
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/train.csv')
test=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/test.csv')
sample_submission=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/sample_submission.csv')

In [3]:
train.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Complaint-Status,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Closed with explanation,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,Closed with non-monetary relief,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,Closed with explanation,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,Closed with explanation,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,Closed with explanation,No,J'ai obtenu un compte de crédit de soins pour ...


In [4]:
train['Transaction-Type'].unique()

array(['Mortgage', 'Credit reporting', 'Bank account or service',
       'Debt collection', 'Credit card', 'Consumer Loan',
       'Credit card or prepaid card',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Student loan', 'Payday loan', 'Vehicle loan or lease',
       'Prepaid card', 'Payday loan, title loan, or personal loan',
       'Money transfer, virtual currency, or money service',
       'Checking or savings account', 'Other financial service',
       'Money transfers', 'Virtual currency'], dtype=object)

In [5]:
train['Complaint-reason'].head()

0    Loan servicing, payments, escrow account
1      Incorrect information on credit report
2                   Using a debit or ATM card
3       Cont'd attempts collect debt not owed
4                              Payoff process
Name: Complaint-reason, dtype: object

In [6]:
len(train['Complaint-reason'].head())

5

In [7]:
train['Company-response'].unique()

array([nan, 'Company chooses not to provide a public response',
       'Company believes it acted appropriately as authorized by contract or law',
       'Company has responded to the consumer and the CFPB and chooses not to provide a public response',
       'Company disputes the facts presented in the complaint',
       'Company believes complaint is the result of an isolated error',
       'Company believes the complaint is the result of a misunderstanding',
       'Company believes complaint represents an opportunity for improvement to better serve consumers',
       'Company believes complaint caused principally by actions of third party outside the control or direction of the company',
       "Company can't verify or dispute the facts in the complaint",
       'Company believes complaint relates to a discontinued policy or procedure'],
      dtype=object)

In [8]:
len(train['Company-response'].unique())

11

In [9]:
train['Consumer-disputes'].unique()

array(['Yes', 'No', nan], dtype=object)

In [10]:
len(train['Consumer-complaint-summary'].unique())

43022

In [11]:
#Distribution of the classes
train['Complaint-Status'].value_counts()

Closed with explanation            34300
Closed with non-monetary relief     5018
Closed with monetary relief         2818
Closed                               809
Untimely response                    321
Name: Complaint-Status, dtype: int64

In [12]:
train[train['Transaction-Type']=='Mortgage']['Complaint-Status'].value_counts()

Closed with explanation            7103
Closed with non-monetary relief     356
Closed with monetary relief         307
Closed                              158
Untimely response                    26
Name: Complaint-Status, dtype: int64

In [13]:
train[train['Transaction-Type']=='Prepaid card']['Complaint-Status'].value_counts()

Closed with explanation            145
Closed with monetary relief        116
Closed with non-monetary relief     47
Name: Complaint-Status, dtype: int64

#Test

In [14]:
test.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary
0,Te-1,8/18/2016,Bank account or service,"Account opening, closing, or management",Company has responded to the consumer and the ...,8/18/2016,No,XXXX / XXXX / 16 I called Citibank to open a c...
1,Te-2,4/18/2016,Debt collection,Communication tactics,Company believes it acted appropriately as aut...,4/20/2016,No,I'm struggling financially. I called and I off...
2,Te-3,3/23/2016,Credit reporting,Incorrect information on credit report,NaN,3/23/2016,No,"In XXXX of 2015, an automatic payment was conf..."
3,Te-4,6/26/2017,Student loan,Dealing with your lender or servicer,NaN,6/26/2017,NaN,"I submitted a request to XXXX, which is my cur..."
4,Te-5,5/13/2016,Credit reporting,Incorrect information on credit report,Company has responded to the consumer and the ...,5/13/2016,No,A state tax lien was filed against me XXXX / X...


In [15]:
train_copy=train.drop(['Complaint-Status'],axis=1)
train_copy.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,No,J'ai obtenu un compte de crédit de soins pour ...


In [16]:
test['Transaction-Type'].unique()

array(['Bank account or service', 'Debt collection', 'Credit reporting',
       'Student loan', 'Consumer Loan', 'Credit card or prepaid card',
       'Mortgage', 'Credit card',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Payday loan', 'Checking or savings account',
       'Payday loan, title loan, or personal loan',
       'Vehicle loan or lease', 'Money transfers',
       'Money transfer, virtual currency, or money service',
       'Prepaid card', 'Other financial service', 'Virtual currency'],
      dtype=object)

In [17]:
#Intersection with Test-----> Transaction-Type

In [18]:
set(test['Transaction-Type'].unique())-(set(train['Transaction-Type'].unique()))

set()

In [19]:
#Intersection with Test---->  Company-response

In [20]:
set(test['Company-response'].unique())-(set(train['Company-response'].unique()))

set()

In [21]:
#Length of the train and test database
print len(train)
print len(test)

43266
18543


In [22]:
#Joining Available Columns in Test and Train for Pre-processing
join=train_copy.append(test,ignore_index=True)
join.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary
0,Tr-1,11/11/2015,Mortgage,"Loan servicing, payments, escrow account",NaN,11/11/2015,Yes,"Seterus, Inc a déposé un faux rapport auprès d..."
1,Tr-2,7/7/2015,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,7/7/2015,No,XX / XX / XXXX La requête en faillite n ° XXXX...
2,Tr-3,5/7/2015,Bank account or service,Using a debit or ATM card,NaN,5/7/2015,No,"El XXXX / XXXX / 15, estaba preparando el vuel..."
3,Tr-4,11/12/2016,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,11/12/2016,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea..."
4,Tr-5,9/29/2016,Credit card,Payoff process,Company has responded to the consumer and the ...,9/29/2016,No,J'ai obtenu un compte de crédit de soins pour ...


In [23]:
#Date Time
join['Date-received'] =  pd.to_datetime(join['Date-received'], format='%m/%d/%Y')
join['Date-sent-to-company'] =  pd.to_datetime(join['Date-sent-to-company'], format='%m/%d/%Y')

In [24]:
len(join['Complaint-reason'].unique())

152

In [25]:
len(join['Company-response'].unique())

11

In [26]:
len(join['Transaction-Type'].unique())

18

In [27]:
#Adding Time Related Columns
#Hour Month DaY Information to data
#join['hour_recvd']= join['Date-received'].apply(lambda x: correct_spelling(x, mispell_dict))
#join['month_recvd']=join['Date-received'].apply(lambda x: correct_spelling(x, mispell_dict))
#join['year_recvd']= join['Date-received'].apply(lambda x: correct_spelling(x, mispell_dict))

join['diff_time']=join['Date-sent-to-company']-join['Date-received']
join['diff_time']=join['diff_time'].apply(lambda x:x.days )

In [28]:
#join.head()
#join['Transaction-Type'].hist()

In [29]:
#Missing Value Treatments
join.isna().any()

Complaint-ID                  False
Date-received                 False
Transaction-Type              False
Complaint-reason              False
Company-response               True
Date-sent-to-company          False
Consumer-disputes              True
Consumer-complaint-summary    False
diff_time                     False
dtype: bool

In [30]:
#Null/Mkssinh
#join.isnull().any()

#Imputing Missing Values for two columns 

In [31]:
#join.isnull().sum()

In [32]:
join['Company-response'].value_counts()

Company has responded to the consumer and the CFPB and chooses not to provide a public response                            15623
Company chooses not to provide a public response                                                                            6149
Company believes it acted appropriately as authorized by contract or law                                                    5409
Company believes the complaint is the result of a misunderstanding                                                           568
Company disputes the facts presented in the complaint                                                                        537
Company believes complaint caused principally by actions of third party outside the control or direction of the company      434
Company believes complaint is the result of an isolated error                                                                416
Company can't verify or dispute the facts in the complaint                                       

In [33]:
join['Consumer-disputes'].value_counts()

No     39487
Yes    11320
Name: Consumer-disputes, dtype: int64

In [34]:
#Imputing Missing Values 
# join['Company-response'].fillna(value=join['Company-response'].value_counts().index[0],inplace =True)
# join['Consumer-disputes'].fillna(value=join['Consumer-disputes'].value_counts().index[0],inplace =True)

In [35]:
#Making Missing as One more Category
join['Company-response'].unique()

array([nan, 'Company chooses not to provide a public response',
       'Company believes it acted appropriately as authorized by contract or law',
       'Company has responded to the consumer and the CFPB and chooses not to provide a public response',
       'Company disputes the facts presented in the complaint',
       'Company believes complaint is the result of an isolated error',
       'Company believes the complaint is the result of a misunderstanding',
       'Company believes complaint represents an opportunity for improvement to better serve consumers',
       'Company believes complaint caused principally by actions of third party outside the control or direction of the company',
       "Company can't verify or dispute the facts in the complaint",
       'Company believes complaint relates to a discontinued policy or procedure'],
      dtype=object)

Making a Separate category for missing

In [36]:
#Company Respose
join['Company-response'].fillna(value='No response',inplace =True)

#Missing Dispute-- No Dispute
join['Consumer-disputes'].fillna(value=join['Consumer-disputes'].value_counts().index[0],inplace =True)

In [37]:
join.isna().any()

Complaint-ID                  False
Date-received                 False
Transaction-Type              False
Complaint-reason              False
Company-response              False
Date-sent-to-company          False
Consumer-disputes             False
Consumer-complaint-summary    False
diff_time                     False
dtype: bool

In [38]:
join.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary,diff_time
0,Tr-1,2015-11-11,Mortgage,"Loan servicing, payments, escrow account",No response,2015-11-11,Yes,"Seterus, Inc a déposé un faux rapport auprès d...",0
1,Tr-2,2015-07-07,Credit reporting,Incorrect information on credit report,Company chooses not to provide a public response,2015-07-07,No,XX / XX / XXXX La requête en faillite n ° XXXX...,0
2,Tr-3,2015-05-07,Bank account or service,Using a debit or ATM card,No response,2015-05-07,No,"El XXXX / XXXX / 15, estaba preparando el vuel...",0
3,Tr-4,2016-11-12,Debt collection,Cont'd attempts collect debt not owed,Company believes it acted appropriately as aut...,2016-11-12,No,"The loan was paid in XXXX XXXX. In XXXX, 4 yea...",0
4,Tr-5,2016-09-29,Credit card,Payoff process,Company has responded to the consumer and the ...,2016-09-29,No,J'ai obtenu un compte de crédit de soins pour ...,0


In [39]:
count_trans=len(join['Transaction-Type'].unique())
print count_trans

18


In [40]:
count_complain_reason=len(join['Complaint-reason'].unique())
print count_complain_reason

152


In [41]:
count_company_response=len(join['Company-response'].unique())
print count_company_response

11


In [42]:
diff_time=np.array(join['diff_time']).reshape(-1,1)

In [43]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
enc_time=scaler.fit_transform(diff_time)

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Processing The categorical Variables

In [44]:
#1.Label Encoding the Categorical Varaible----TransactionType
le1= preprocessing.LabelEncoder()
enc_trans_type=le1.fit_transform(join['Transaction-Type'])

In [45]:
#2.Label Encoding the Categorical Varaible----Complaint-reason
le2= preprocessing.LabelEncoder()
enc_complain_reason=le2.fit_transform(join['Complaint-reason'])

In [46]:
#3.Label Encoding the Categorical Varaible----Company-response
le3= preprocessing.LabelEncoder()
enc_comp_response=le3.fit_transform(join['Company-response'])

In [47]:
#4.Label Encoding the Categorical Varaible----Consumer-disputes
le4= preprocessing.LabelEncoder()
enc_cons_dispute=le4.fit_transform(join['Consumer-disputes'])

In [48]:
#Response 
le5= preprocessing.LabelEncoder()
y_cat=le5.fit_transform(train['Complaint-Status'])

In [49]:
from keras.utils import to_categorical
y_enc=to_categorical(y_cat,num_classes=5)

Processing the Text Data Column----> Consumer-complaint-summary

In [50]:
a=join['Consumer-complaint-summary'][100]
a

'Esta escuela se ha cerrado permanentemente y no puedo obtener un registro de esta escuela ni completar mi educaci\xc3\xb3n.'

In [51]:
#import re
#re.sub(r'[^a-zA-Z]', ' ', a)

In [52]:
from googletrans import Translator
translator = Translator()


In [53]:
# #Translating
# p1=p.text
# p1

# #Removing Numbers and other things
# q1=re.sub(r'[^a-zA-Z]', ' ', p1)
# q1

# #Replacing XXX
# r1=q1.replace('XX'," ")
# r1

# #Replacing Extra spaces
# s1=re.sub('\s+', ' ', r1).strip()
# s1

# #Lower case for word vectors
# s1.lower()

In [54]:
#join['Consumer-complaint-summary'] = join['Consumer-complaint-summary'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))

In [55]:
train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
test['Consumer-complaint-summary'] = test['Consumer-complaint-summary'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))


In [56]:
train['Consumer-complaint-summary'] = train['Consumer-complaint-summary'].apply(lambda x: str(x))
test['Consumer-complaint-summary'] = test['Consumer-complaint-summary'].apply(lambda x: str(x))

In [57]:
train['Consumer-complaint-summary']=train['Consumer-complaint-summary'].str.replace('XX','')
test['Consumer-complaint-summary']=test['Consumer-complaint-summary'].str.replace('XX','')

In [58]:
train['Consumer-complaint-summary']=train['Consumer-complaint-summary'].apply(lambda x:re.sub('\s+', ' ', x).strip())
test['Consumer-complaint-summary']=test['Consumer-complaint-summary'].apply(lambda x:re.sub('\s+', ' ', x).strip())

In [59]:
train['Consumer-complaint-summary']=train['Consumer-complaint-summary'].apply(lambda x:x.lower())
test['Consumer-complaint-summary']=test['Consumer-complaint-summary'].apply(lambda x:x.lower())

In [60]:
max_features = 100000
tokenizer = Tokenizer(lower = True, filters='', num_words=max_features)
full_text = list(join['Consumer-complaint-summary'].values)
tokenizer.fit_on_texts(full_text)

In [61]:
#Tokenizing the documents---- convert to strings
train_tokenized = tokenizer.texts_to_sequences(train['Consumer-complaint-summary'].fillna('missing'))
test_tokenized=   tokenizer.texts_to_sequences(test ['Consumer-complaint-summary'].fillna('missing'))

In [62]:
#Padding the same
max_len = 850
X_train = pad_sequences(train_tokenized, maxlen = max_len,padding='post')
X_test = pad_sequences(test_tokenized, maxlen = max_len,padding='post')

In [63]:
#word_index is dictionary of the words and the sequence
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 210800 unique tokens.


In [64]:
np.random.seed(0)
x = np.random.random(100)

In [65]:
#Embedding matrix creation
nb_words = min(100000, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, 100))

for word, i in word_index.items():
    #print i
    if i >= nb_words:
        continue
    embedding_vector = x
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = x
        
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 1


In [66]:
#Transaction
enc_trans_type_train=enc_trans_type[0:43266]
enc_trans_type_test=enc_trans_type[43266:]

In [67]:
#Complaint reason
enc_complain_reason_train=enc_complain_reason[0:43266]
enc_complain_reason_test=enc_complain_reason[43266:]

In [68]:
enc_comp_response_train=enc_comp_response[0:43266]
enc_comp_response_test= enc_comp_response[43266:]

In [69]:
enc_cons_dispute_train=enc_cons_dispute[0:43266]
enc_cons_dispute_test= enc_cons_dispute[43266:]

In [70]:
enc_time_train=enc_time[0:43266]
enc_time_test=enc_time[43266:]

In [71]:
#words[1000:2000]

In [72]:
def model_lstm():
    
    #Defining the input-----> Transaction
    inputs1 = Input(shape=(1,))
    layer_transaction = Embedding(count_trans , 5 ,input_length=1)(inputs1)
    layer_transaction = Flatten()(layer_transaction)
    
    #Defining the input-----> Complaint-reason
    inputs2 = Input(shape=(1,))
    layer_complain_reason = Embedding(count_complain_reason, 50, input_length=1)(inputs2)
    layer_complain_reason = Flatten()(layer_complain_reason)
    
    #Defining the input-----> Complaint-reason
    inputs3 = Input(shape=(1,))
    layer_company_response = Embedding(count_company_response ,5,input_length=1)(inputs3)
    layer_company_response = Flatten()(layer_company_response)
    
    #Defining the input-----> Consumer-disputes
    inputs4 = Input(shape=(1,))
    layer_consumer_disputes = Embedding(2 ,2,input_length=1)(inputs4)
    layer_consumer_disputes = Flatten()(layer_consumer_disputes)
    
    #Defining the input ----> Time
    inputs5 = Input(shape=(1,))
    
    #Defining the input-----> Consumer-complaint-summary
    inputs6 = Input(shape=(max_len,))
    layer_complaint =  Embedding(nb_words, 100, input_length=max_len, trainable=True)(inputs6)
    
    #LSTM
    layer_lstm = Bidirectional(LSTM(128, return_sequences=False))(layer_complaint)
    
    #Merge
    layer_merge= concatenate([layer_transaction, 
                              layer_complain_reason, 
                              layer_company_response, 
                              layer_consumer_disputes,
                              inputs5, 
                              layer_lstm], axis=1)
    
    #Dense Layers
    layer_dense = Dense(64, activation='relu')(layer_merge)
    layer_dense = Dropout(0.3)(layer_dense)
    
    #Output Layer
    probabilities = Dense(5,activation='softmax')(layer_dense)

    model = Model(inputs=[inputs1, inputs2, inputs3, inputs4, inputs5, inputs6], outputs=probabilities)
    return model

    

In [73]:
model=model_lstm()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_6 (I

In [74]:
model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr=0.001), metrics = ["accuracy"])

In [75]:
history = model.fit([enc_trans_type_train,
                     enc_complain_reason_train,
                     enc_comp_response_train,
                     enc_cons_dispute_train,
                     enc_time_train,
                     X_train] ,y_enc, batch_size = 512, epochs = 2, validation_split=0.1, 
                     verbose = 1)


Train on 38939 samples, validate on 4327 samples
Epoch 1/2
38939/38939 [==============================] - 1764s 45ms/step - loss: 0.8170 - acc: 0.7815 - val_loss: 0.6770 - val_acc: 0.7948
Epoch 2/2
38939/38939 [==============================] - 983s 25ms/step - loss: 0.6501 - acc: 0.7935 - val_loss: 0.6285 - val_acc: 0.7976


In [ ]:
#Tets
y_prob = model.predict([enc_trans_type_test,enc_complain_reason_test,
                     enc_comp_response_test,enc_cons_dispute_test,enc_time_test,X_train]) 

#Pred
y_classes = y_prob.argmax(axis=-1)

In [ ]:
y_pred=list(le5.inverse_transform(y_classes))

In [ ]:
submission=pd.DataFrame()
submission['Complaint-ID']=test['Complaint-ID']
submission['Complaint-Status']=y_pred

In [ ]:
submission.to_csv(path_or_buf="/Users/s0c02nj/Desktop/Submission_BRAINWAVES1.csv", encoding='utf-8',index=False)